In [36]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import spatial
from sklearn.manifold import TSNE
import zipfile
from tqdm import tqdm

In [5]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%load_ext autoreload
%autoreload 2

from src.path import get_path_from_root

In [45]:
embeddings_dict = {}
with zipfile.ZipFile(str(get_path_from_root('data/search/glove.6B.csv')), 'r') as archive:
    with archive.open("glove.6B.50d.txt", 'r') as f:
        for line in tqdm(f, total=400000):
            line = str(line, encoding='utf-8')
            parts = line.strip().split(' ')
            word = parts[0]
            vec = np.asarray(parts[1:], "float32")
            embeddings_dict[word] = vec

100%|██████████| 400000/400000 [00:07<00:00, 50705.29it/s]


In [47]:
embeddings_dict['walrus']

array([ 0.48698 , -0.45614 , -0.88501 , -0.58914 ,  0.75248 ,  0.1977  ,
       -0.31347 , -0.67976 , -0.25581 , -0.86747 ,  0.95085 ,  0.62886 ,
        0.98183 ,  0.68557 ,  0.38731 ,  0.63174 ,  0.70197 ,  0.85176 ,
       -0.86779 ,  0.6683  , -0.77849 , -0.23294 ,  0.68584 , -0.50269 ,
        0.39149 , -0.11991 , -0.58507 ,  1.0757  ,  0.55094 , -0.456   ,
       -0.66728 , -0.16855 , -0.18119 ,  1.4549  , -0.50935 ,  0.40332 ,
       -0.28225 ,  0.070051, -0.71228 , -0.33878 ,  0.34351 , -0.037361,
        0.47597 , -0.57553 ,  0.73109 , -0.43637 ,  0.27931 ,  0.4762  ,
       -0.86048 , -0.20388 ], dtype=float32)